# Step 1 Dev Set Labels

In [ ]:
def step1_dev_set_labels(*, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedProductRating import dev_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    taskllm_model_id = f'User_keq{k}_t5base_step1_iter{step1_iter}'
    t5.load_adapter(taskllm_model_id, 'taskllm')
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]
    
    print(f'*** step1_iter: {step1_iter} ***')

    devgolds = []
    with ProgressPrinter(f'{k} MAE (dev)', f'{k} MSE (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for examples, labels in dev:
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['review'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                indices = [ torch.topk(embeddings[a,:] @ embeddings[a+1:b,:].T, dim=0, k=safek).indices
                            for a, b in zip(splits, splits[1:])
                            for safek in (max(0, min(k, b-a-1)),)
                          ]
                prompts = [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in index.to('cpu').tolist() ])
                            for ex, index in zip(examples, indices) ]
                cumul = taskllm.predict(prompts).exp().cumsum(dim=1)
                guesses = (cumul>=0.5).long().argmax(dim=1)
                targets = [ int(label) - 1 for label in labels ]
                targets = torch.Tensor(targets).long().to(guesses.device)
                mae = torch.abs(guesses - targets).float().mean().item()
                mse = torch.square(guesses - targets).float().mean().item()

                for ex, guess in zip(examples, guesses):
                    devgolds.append({ 'id': ex['id'], 'output': f'{1+guess}' })

            printer.addobs(mae, mse)

    with open(f'lamp3u_step1_dev_golds.json', 'w') as jsonfile:
        json.dump({ 'task': 'LaMP_2', 'golds': devgolds }, jsonfile)
            
step1_dev_set_labels(k=4, step1_iter='2_augment2')

*** step1_iter: 2_augment2 ***
n       4 MAE (dev) (since) 4 MSE (dev) (since)      dt
1             0.500 (0.500)       0.500 (0.500)  1.99 s
2             0.312 (0.125)       0.312 (0.125)  3.85 s
4             0.281 (0.250)       0.281 (0.250)  6.02 s
8             0.266 (0.250)       0.266 (0.250)  13.2 s
16            0.312 (0.359)       0.359 (0.453)  24.8 s
32            0.281 (0.250)       0.336 (0.312)  50.1 s
64            0.283 (0.285)       0.404 (0.473)  1.62 m
128           0.256 (0.229)       0.332 (0.260)  3.26 m
256           0.244 (0.231)       0.309 (0.286)  6.55 m
313           0.247 (0.261)       0.312 (0.327)  8.03 m


In [ ]:
def step2_dev_set_labels(*, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedProductRating import dev_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.load_adapter(f'User_keq{k}_t5base_step2_iter{step2_iter}', 'rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat", model_id=f'User_keq{k}_t5base_step2_iter{step2_iter}')

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0], 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=k).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} ***')
    nvoters = 1
    
    with ProgressPrinter(f'{k} MAE (dev)', f'{k} MSE (dev)') as printer:
        devgolds = []
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for examples, labels in dev:
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['review'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                predicts = torch.cat(inner_batch(func = taskllm.predict,
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []),)
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ (predicts[a:b,:].logsumexp(dim=0, keepdim=True).exp().cumsum(dim=1) >= 0.5 * (b-a)).long().argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                targets = [ int(label) - 1 for label in labels ]
                targets = torch.Tensor(targets).long().to(guesses.device)
                mae = torch.abs(guesses - targets).float().mean().item()
                mse = torch.square(guesses - targets).float().mean().item()

                for ex, guess in zip(examples, guesses):
                    devgolds.append({ 'id': ex['id'], 'output': f'{1+guess}' })

            printer.addobs(mae, mse)

        printer.print()
        printer.autoprint = False

    with open(f'lamp3u_step2_dev_golds.json', 'w') as jsonfile:
        json.dump({ 'task': 'LaMP_2', 'golds': devgolds }, jsonfile)

step2_dev_set_labels(k=4, step1_iter='2_augment2', step2_iter='0_augment2')

*** step1_iter: 2_augment2 step2_iter: 0_augment2 ***
n       4 MAE (dev) (since) 4 MSE (dev) (since)      dt
1             0.375 (0.375)       0.375 (0.375)  5.26 s
2             0.250 (0.125)       0.250 (0.125)  10.2 s
4             0.219 (0.188)       0.219 (0.188)  18.5 s
8             0.234 (0.250)       0.234 (0.250)  36.9 s
16            0.312 (0.391)       0.359 (0.484)  1.25 m
32            0.297 (0.281)       0.344 (0.328)   2.5 m
64            0.293 (0.289)       0.410 (0.477)  4.91 m
128           0.270 (0.246)       0.348 (0.285)  9.87 m
256           0.252 (0.234)       0.317 (0.287)  19.7 m
313           0.254 (0.263)       0.322 (0.342)  24.2 m
